In [94]:
import pandas as pd
import time
import re
from selenium import webdriver
from datetime import datetime

## 종목별 2011.12.31 ~ 2021.12.31별 네이버 금융 종목 기사 뉴스 헤드라인 크롤링 
- 일자별 기사의 개수가 모두 다르기 때문에 1page에 있는 일별 최근 기사 기준 20개를 크롤링
- 일별 기사가 20개가 안되는 경우 ->  기사가 총 10 ~ 20개일 경우는 10개만, 5 ~ 10개는 5개, 1 ~ 5개는 1개만 크롤링 진행


In [95]:
# 기간 내의 날짜 리스트 생성
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [date.strftime("%Y-%m-%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates
    
dates = date_range("2011-12-31", "2021-12-31")

In [96]:
# '네이버 금융' 페이지 접근
url = 'https://finance.naver.com/news'
driver = webdriver.Chrome('C:/Users/seonwook/Documents/ds_study/driver/chromedriver.exe') # Windows: '../driver/chromedriver.exe'
driver.get(url)
driver.maximize_window()

### 종목 선정 기준
- 한국 거래소 시가총액 기준 1위~10위
- 단, 동일 계열사는 제외하고 다음 순위에 있는 기업으로 선정 (예시: 선정: 카카오 제외: 카카오페이, 카카오뱅크)

In [97]:
# 종목 선정 (한국 거래소 기준)
stock_list = [] # 종목리스트 선정  현대차 005380
temp_list = [] 
for stock in stock_list:
    # 종목 뉴스 검색
    some_tag = driver.find_element_by_css_selector('#newsMainTop > div > div.inner_area_right > form > div > input')
    some_tag.clear()
    some_tag.send_keys('{}'.format(stock)) # 종목명 입력

    some_tag = driver.find_element_by_css_selector('#newsMainTop > div > div.inner_area_right > form > div > a > input[type=image]')
    some_tag.click()
    time.sleep(3)
    
    # 뉴스 헤드라인, 날짜 가져오기
    main = driver.current_url
    for date in dates:
        url = main + '&stDateStart={}&stDateEnd={}'.format(date, date)
        driver.get(url)
        temp = driver.find_element_by_css_selector('#contentarea_left > div.newsSchResult > dl')
        temp = temp.text
        temp = re.split("[\n|]", temp )
        if len(temp) < 15 and len(temp) >=3:
            for j in range(0, 3, 3):
                temp_list.append([stock, temp[j], temp[j+2]])
        
        if len(temp) < 30 and len(temp) >=15:
            for j in range(0, 15, 3):
                temp_list.append([stock, temp[j], temp[j+2]])

        if len(temp) < 60 and len(temp) >= 30:
            for j in range(0, 30, 3):
                temp_list.append([stock, temp[j], temp[j+2]])

        if len(temp) >= 60:
            for j in range(0, 60, 3):
                temp_list.append([stock, temp[j], temp[j+2]])
driver.quit()    

In [98]:
# 설정한 기간동안의 총 기사 개수
len(temp_list)

33126

In [99]:
temp_list[:5] # 종목, 헤드라인, 일자 형식

[['현대차', '[2012 증시는?] 20개 증권사 전문가들이 꼽은 2012 유망주', ' 2011-12-31 17:00'],
 ['현대차', '잘 나가는 기업, 못 나가는 기업···"다 이유가 있다"', ' 2011-12-31 16:48'],
 ['현대차', '4대그룹 "새해 어렵지만 성장 드라이브"', ' 2011-12-31 13:01'],
 ['현대차', '[2012 증시는?] 미래에셋 "상저하고, 1년이상 장기투자를"', ' 2011-12-31 10:01'],
 ['현대차', '[2012] 삼성證, 글로벌 정책과 유동성 환경에 좌우될 것', ' 2011-12-31 09:49']]

In [100]:
# 컬럼명 지정: pd.DataFrame형식으로 바꾸기 위해
col_name = ['종목', '헤드라인', '날짜']
df_stock = pd.DataFrame(temp_list, columns=col_name)

In [101]:
df_stock.head()

,종목,헤드라인,날짜
0,현대차,[2012 증시는?] 20개 증권사 전문가들이 꼽은 2012 유망주,2011-12-31 17:00
1,현대차,"잘 나가는 기업, 못 나가는 기업···""다 이유가 있다""",2011-12-31 16:48
2,현대차,"4대그룹 ""새해 어렵지만 성장 드라이브""",2011-12-31 13:01
3,현대차,"[2012 증시는?] 미래에셋 ""상저하고, 1년이상 장기투자를""",2011-12-31 10:01
4,현대차,"[2012] 삼성證, 글로벌 정책과 유동성 환경에 좌우될 것",2011-12-31 09:49


In [102]:
# 크롤링해온 날짜 형식 전처리
for i in range(len(df_stock)):
    df_stock['날짜'][i] = df_stock['날짜'][i][:11].lstrip()

In [103]:
df_stock.head()

,종목,헤드라인,날짜
0,현대차,[2012 증시는?] 20개 증권사 전문가들이 꼽은 2012 유망주,2011-12-31
1,현대차,"잘 나가는 기업, 못 나가는 기업···""다 이유가 있다""",2011-12-31
2,현대차,"4대그룹 ""새해 어렵지만 성장 드라이브""",2011-12-31
3,현대차,"[2012 증시는?] 미래에셋 ""상저하고, 1년이상 장기투자를""",2011-12-31
4,현대차,"[2012] 삼성證, 글로벌 정책과 유동성 환경에 좌우될 것",2011-12-31


## 해당 종목의 일자별 주가 변동에 대한 파일과 병합
- 기사가 주식 가격에 미치는 영향을 토대로 긍/부정 라벨링을 하기 위해 해당 종목의 일자별 주가 급락에 대한 데이터를 불러옴
- 크롤링해온 종목별 기사 데이터와 병합

In [104]:
change_rate = pd.read_csv('C:/Users/seonwook/Documents/ten_change.csv', index_col=0)
change_rate.head()

,date,change,name
0,2012-01-02,1,삼성전자
1,2012-01-03,1,삼성전자
2,2012-01-04,0,삼성전자
3,2012-01-05,0,삼성전자
4,2012-01-06,0,삼성전자


In [105]:
# 병합을 위해 컬럼명 통일
df_stock.columns = ['name', 'headline', 'date']

In [106]:
df_stock.head()

,name,headline,date
0,현대차,[2012 증시는?] 20개 증권사 전문가들이 꼽은 2012 유망주,2011-12-31
1,현대차,"잘 나가는 기업, 못 나가는 기업···""다 이유가 있다""",2011-12-31
2,현대차,"4대그룹 ""새해 어렵지만 성장 드라이브""",2011-12-31
3,현대차,"[2012 증시는?] 미래에셋 ""상저하고, 1년이상 장기투자를""",2011-12-31
4,현대차,"[2012] 삼성證, 글로벌 정책과 유동성 환경에 좌우될 것",2011-12-31


### inner join을 통해 공통된 부분만 추출하도록 병합
- 주식장은 주말을 제외한 평일에만 개장한다는 점을 고려

In [107]:
df_stock = pd.merge(df_stock, change_rate, on=['name', 'date'], how='inner')

In [108]:
df_stock.head()

,name,headline,date,change
0,현대차,재계 총수들의 올해 첫 화두는 ‘위기’ ‘투자’ ‘인재’,2012-01-02,0
1,현대차,[경제 굿&노굿] CEO들 신년사로 보는 2012 경제전망,2012-01-02,0
2,현대차,[경제 굿&노굿] CEO들 신년사로 보는 2012 경제전망,2012-01-02,0
3,현대차,재계 빅3 공격적 투자‥나머지는 '흐림',2012-01-02,0
4,현대차,“공생발전·내실경영으로 글로벌 위기 돌파”,2012-01-02,0


In [109]:
len(df_stock)

24275

## csv로 저장

In [110]:
df_stock.to_csv('C:/Users/seonwook/Documents/{}.csv'.format(df_stock['name'][0]), encoding='utf-8-sig') # 한글 깨짐 -> utf-8-sig

## 데이터 불러와보기

In [111]:
temp = pd.read_csv('C:/Users/seonwook/Documents/{}.csv'.format(df_stock['name'][0]), encoding='utf-8-sig', index_col=0)
temp.head()

,name,headline,date,change
0,현대차,재계 총수들의 올해 첫 화두는 ‘위기’ ‘투자’ ‘인재’,2012-01-02,0
1,현대차,[경제 굿&노굿] CEO들 신년사로 보는 2012 경제전망,2012-01-02,0
2,현대차,[경제 굿&노굿] CEO들 신년사로 보는 2012 경제전망,2012-01-02,0
3,현대차,재계 빅3 공격적 투자‥나머지는 '흐림',2012-01-02,0
4,현대차,“공생발전·내실경영으로 글로벌 위기 돌파”,2012-01-02,0


In [112]:
temp2 = pd.read_csv('C:/Users/seonwook/Documents/hyundai.csv', index_col=0)
temp2.head()

,name,headline,date,change
0,현대차,[심층기획] 3500억대 귀속자산… 전경련 해체 시 향방 주목,2017-01-02,1
1,현대차,[심층기획] 정경유착 오명에 줄줄이 탈퇴… 존폐 기로에 선 전경련,2017-01-02,1
2,현대차,"현대차株, 올해 내수 줄어도 신흥국 전망 '맑음'",2017-01-02,1
3,현대차,특검 수사에 '바짝 낮춘 이재용'…최태원은 '대담하게',2017-01-02,1
4,현대차,"화성남자 vs 금성여자 ""이 차가 최고라니까!""",2017-01-02,1


In [113]:
len(temp), len(temp2)

(24275, 24508)

In [114]:
temp = pd.concat([temp, temp2], ignore_index=True)
len(temp)

48783

In [115]:
temp.to_csv('C:/Users/seonwook/Documents/현대자동차.csv', encoding='utf-8-sig')